# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

733942

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

taxes_communale


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

2

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences[78]]

CPU times: user 230 µs, sys: 10 µs, total: 240 µs
Wall time: 262 µs


['maladie',
 ',',
 'le',
 'conseil',
 'ajourne',
 'l',
 "'",
 'examen',
 'de',
 'la',
 'reclamation',
 'des',
 'tailleurs',
 'de',
 'pierre',
 'et',
 'marchands',
 'de',
 'briques',
 ',',
 'contre',
 'quelques',
 'articles',
 'des',
 'taxes_communales',
 ',',
 'et',
 'la',
 'discussion',
 'des',
 'budgets',
 'de',
 'la']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences[78]]

CPU times: user 68 µs, sys: 3 µs, total: 71 µs
Wall time: 84.9 µs


['maladie',
 ',',
 'le',
 'conseil',
 'ajourne',
 'l',
 "'",
 'examen',
 'de',
 'la',
 'reclamation',
 'des',
 'tailleurs',
 'de',
 'pierre',
 'et',
 'marchands',
 'de',
 'briques',
 ',',
 'contre',
 'quelques',
 'articles',
 'des',
 'taxes_communales',
 ',',
 'et',
 'la',
 'discussion',
 'des',
 'budgets',
 'de',
 'la']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [17]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de'], ['bruxelles'], ['bulletin', 'ires', '8eanas'], ['dl', '!'], ['conseil'], ['communal'], ['annee'], ['1847', '.'], ['au'], ['ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin'], ['conseil'], ['aes'], ['seances'], ['communal', '.'], ['annee'], ['1847', '.'], ['bruxelles', ','], ['imprimerie'], ['d', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ','], ['rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ','], ['1', '84', '8'], ['de', '!'], ['du', 'consei'], ['dibi'], ['e', '.', '-', 'communication'], ['conclusions', 'de', 'la', 'section', 'des'], ['du', 'nouvel_hospice', 'pour', 'les', 'av'], ['enraisonde', 'l', "'", 'absence', '&'], ['maladie', '.', 'le', 'conseil', 'ajourne'], ['leurs', 'de', 'pierre', 'el', 'marchai'], ['cles', 'des', 'taxes', 'communale', "'"], ['bieniaance', 'eldeseianv'], ['il', 'est', 'donne', 'communie', ';'], ['mandant', 'le', 'o', 'p', 'fa'], ['gnant

## Entrainement d'un modèle Word2Vec sur ce corpus

In [18]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 47.8 s, sys: 1.09 s, total: 48.9 s
Wall time: 24.7 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [19]:
outfile = f"../data/bulletins.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [20]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [21]:
model.wv["bruxelles"]

array([-1.4484682 , -2.029069  ,  0.08226616,  0.48995095,  0.08995774,
       -2.9451852 , -0.5143855 ,  0.8510913 , -3.0063484 , -0.2808212 ,
       -2.1898217 ,  1.7858124 ,  0.6347236 ,  2.7516363 , -0.30383784,
        3.2109025 ,  1.4884175 , -0.72725403,  0.8394891 , -3.4555738 ,
        0.39432156, -1.0292234 ,  1.1174304 ,  0.16509397, -1.6287484 ,
        0.6629327 , -2.1525369 , -1.6429336 ,  0.6327619 , -0.65064985,
       -2.0462487 , -1.143783  ], dtype=float32)

### Calculer la similarité entre deux termes

In [30]:
model.wv.similarity("maladie", "nettoyage")

0.19724318

### Chercher les mots les plus proches d'un terme donné

In [32]:
model.wv.most_similar("culture", topn=20)

[('sellette', 0.9083437919616699),
 ('son_profit', 0.8935544490814209),
 ('fer_battu', 0.8909679651260376),
 ('cuir', 0.8741768002510071),
 ('galerie_souterraine', 0.8734057545661926),
 ('justice_distributive', 0.8678615093231201),
 ('vase', 0.8677728176116943),
 ('les_eaux_pluviales', 0.8600560426712036),
 ('enfant_trouve', 0.8581166863441467),
 ('deux_series', 0.8571169972419739),
 ('trois_annees', 0.8555673360824585),
 ('perspective', 0.8547520041465759),
 ('banquette', 0.8514435887336731),
 ('fenetre', 0.8506179451942444),
 ('annee_1849', 0.8500445485115051),
 ('madame_marx', 0.8490403890609741),
 ('plein_vent', 0.8489614129066467),
 ('rouges', 0.847305178642273),
 ('chaleur', 0.8467389345169067),
 ('butte', 0.8462693691253662)]

### Faire des recherches complexes à travers l'espace vectoriel

In [24]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.7762062549591064), ('son_domicile', 0.7383571267127991), ('paris', 0.653240442276001), ('gand', 0.6526199579238892), ('basse', 0.6490806937217712), ('nivelles', 0.6161742210388184), ('londres', 0.6140671968460083), ('retribues', 0.6121885776519775), ('thotel', 0.607509970664978), ('saint_-_gilles', 0.5998967289924622)]
